In [1]:
import arviz as az
import matplotlib.pyplot as plt
import pandas as pd

import jax.numpy as jnp
from jax import random

import numpyro
from numpyro.diagnostics import hpdi, print_summary, summary
import numpyro.distributions as dist
import numpyro.optim as optim
from numpyro.infer import Predictive, SVI, Trace_ELBO
from numpyro.infer.autoguide import AutoLaplaceApproximation, AutoNormal

Qn1. The first two problems are based on the same data. The data in data(foxes)
are 116 foxes from 30 different urban groups in England. These fox groups
are like street gangs. Group size (groupsize) varies from 2 to 8 individuals.
Each group maintains its own (almost exclusive) urban territory. Some territories are larger than others. The area variable encodes this information.
Some territories also have more avgfood than others. And food influences
the weight of each fox. Assume this DAG:

A --> F --> G; F--> W; G-->W

where F is avgfood, G is groupsize, A is area, and W is weight.
Use the backdoor criterion and estimate the total causal influence of A on
F. What effect would increasing the area of a territory have on the amount
of food inside it?

In [2]:
foxes_df = pd.read_csv('../Data/foxes.csv', delimiter = ";")
foxes_df.head(3)

,group,avgfood,groupsize,area,weight
0,1,0.37,2,1.09,5.02
1,1,0.37,2,1.09,2.84
2,2,0.53,2,2.05,5.33


In [3]:
foxes_df.describe()

,group,avgfood,groupsize,area,weight
count,116.000000,116.000000,116.000000,116.000000,116.000000
mean,17.206897,0.751724,4.344828,3.169138,4.529655
std,8.002736,0.198316,1.538511,0.928354,1.184023
min,1.000000,0.370000,2.000000,1.090000,1.920000
25%,11.750000,0.660000,3.000000,2.590000,3.720000
50%,18.000000,0.735000,4.000000,3.130000,4.420000
75%,24.000000,0.800000,5.000000,3.772500,5.375000
max,30.000000,1.210000,8.000000,5.070000,7.550000


In [4]:
# foxes_normalized_df=(foxes_df-foxes_df.mean())/foxes_df.std()
foxes_normalized_df = foxes_df
# standardize variables
standardize = lambda x: (x - x.mean()) / x.std()
foxes_normalized_df["avgfood"] = foxes_df.avgfood.pipe(standardize)
foxes_normalized_df["area"] = foxes_df.area.pipe(standardize)

In [5]:
foxes_normalized_df.describe()

,group,avgfood,groupsize,area,weight
count,116.000000,1.160000e+02,116.000000,1.160000e+02,116.000000
mean,17.206897,3.024401e-16,4.344828,-6.393353e-16,4.529655
std,8.002736,1.000000e+00,1.538511,1.000000e+00,1.184023
min,1.000000,-1.924829e+00,2.000000,-2.239596e+00,1.920000
25%,11.750000,-4.625154e-01,3.000000,-6.238331e-01,3.720000
50%,18.000000,-8.433082e-02,4.000000,-4.215842e-02,4.420000
75%,24.000000,2.434292e-01,5.000000,6.499268e-01,5.375000
max,30.000000,2.310838e+00,8.000000,2.047562e+00,7.550000


In [6]:
def model(area, avgfood=None):
    a = numpyro.sample("a", dist.Normal(0, 0.2))
    b = numpyro.sample("b", dist.Normal(0, 0.5))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    mu = numpyro.deterministic("mu", a + b * area)
    numpyro.sample("avgfood", dist.Normal(mu, sigma), obs=avgfood)

m1 = AutoLaplaceApproximation(model)
svi = SVI(
    model,
    m1,
    optim.Adam(1),
    Trace_ELBO(),
    area=foxes_normalized_df.area.values,
    avgfood=foxes_normalized_df.avgfood.values,
)
svi_result = svi.run(random.PRNGKey(0), 1000)
p1 = svi_result.params

100%|██████████| 1000/1000 [00:00<00:00, 1802.65it/s, init loss: 801.0769, avg. loss [951-1000]: 78.9178]


In [7]:
post = m1.sample_posterior(random.PRNGKey(1), p1, (1000,))
post.pop('mu')
print_summary(post, 0.89, False)


                mean       std    median      5.5%     94.5%     n_eff     r_hat
         a      0.02      0.04      0.03     -0.05      0.09    931.50      1.00
         b      0.88      0.05      0.87      0.81      0.95   1109.99      1.00
     sigma      0.48      0.03      0.48      0.43      0.54    946.36      1.00



Answer: given that "b" is a relatively high value with a small spread, f = a + b*Area. Increasing Area will have a strong effect on food. 

Qn2. Now infer both the total and direct causal effects of adding food F to a
territory on the weight Wof foxes. Which covariates do you need to adjust for
in each case? In light of your estimates from this problem and the previous
one, what do you think is going on with these foxes? Feel free to speculate—
all that matters is that you justify your speculation.

In [8]:
# normalize other columns 
foxes_normalized_df["weight"] = foxes_df.weight.pipe(standardize)
foxes_normalized_df["groupsize"] = foxes_df.groupsize.pipe(standardize)

In [10]:
# 2.1 total effect 
def model(avgfood, weight=None):
    a = numpyro.sample("a", dist.Normal(0, 0.2))
    b = numpyro.sample("b", dist.Normal(0, 0.5))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    mu = numpyro.deterministic("mu", a + b * avgfood)
    numpyro.sample("avgfood", dist.Normal(mu, sigma), obs=weight)

m2_1 = AutoLaplaceApproximation(model)
svi = SVI(
    model,
    m2_1,
    optim.Adam(1),
    Trace_ELBO(),
    avgfood=foxes_normalized_df.avgfood.values,
    weight=foxes_normalized_df.weight.values,
)
svi_result = svi.run(random.PRNGKey(0), 1000)
p2_1 = svi_result.params

100%|██████████| 1000/1000 [00:00<00:00, 2253.54it/s, init loss: 3382.3159, avg. loss [951-1000]: 164.5948]


In [13]:
post = m2_1.sample_posterior(random.PRNGKey(1), p2_1, (1000,))
post.pop('mu')
print_summary(post, 0.89, False)


                mean       std    median      5.5%     94.5%     n_eff     r_hat
         a      0.01      0.08      0.01     -0.14      0.13    931.50      1.00
         b     -0.03      0.09     -0.03     -0.16      0.13   1111.41      1.00
     sigma      1.00      0.07      1.00      0.90      1.11    949.31      1.00



Ans: The total effect of food on weight is small as b is tiny. 

In [14]:
# 2.2 direct effect 
def model(avgfood, groupsize,weight=None):
    a = numpyro.sample("a", dist.Normal(0, 0.2))
    bF = numpyro.sample("bF", dist.Normal(0, 0.5))
    bG = numpyro.sample("bG", dist.Normal(0, 0.5))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    mu = numpyro.deterministic("mu", a + bF * avgfood + bG*groupsize)
    numpyro.sample("avgfood", dist.Normal(mu, sigma), obs=weight)

m2_2 = AutoLaplaceApproximation(model)
svi = SVI(
    model,
    m2_2,
    optim.Adam(1),
    Trace_ELBO(),
    avgfood=foxes_normalized_df.avgfood.values,
    groupsize=foxes_normalized_df.groupsize.values,
    weight=foxes_normalized_df.weight.values,
)
svi_result = svi.run(random.PRNGKey(0), 1000)
p2_2 = svi_result.params

100%|██████████| 1000/1000 [00:00<00:00, 2044.76it/s, init loss: 1565.1736, avg. loss [951-1000]: 160.0162]


In [15]:
post = m2_2.sample_posterior(random.PRNGKey(1), p2_2, (1000,))
post.pop('mu')
print_summary(post, 0.89, False)


                mean       std    median      5.5%     94.5%     n_eff     r_hat
         a     -0.00      0.08     -0.01     -0.13      0.11   1049.96      1.00
        bF      0.48      0.18      0.48      0.19      0.78    822.38      1.00
        bG     -0.58      0.18     -0.58     -0.88     -0.31    836.43      1.00
     sigma      0.95      0.06      0.95      0.84      1.03    958.43      1.00



Ans: food has a positive and moderate effect on weight. However, the effect can vary. 
Interestingly, the direct effect of groupsize on weight is negative. One possibility is that within larger groups there is more compeition over the same amount of food, so there's less or same amount of food per wolf. Similarly, the increased in food could have resulted in increase in number of wolfs instead of weight per food. To test, we can look into the relationship between food and groupsize. 

In [18]:
# 2.3 food and group size
def model(avgfood, groupsize=None):
    a = numpyro.sample("a", dist.Normal(0, 0.2))
    b = numpyro.sample("b", dist.Normal(0, 0.5))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    mu = numpyro.deterministic("mu", a + b * avgfood)
    numpyro.sample("avgfood", dist.Normal(mu, sigma), obs=groupsize)

m2_3 = AutoLaplaceApproximation(model)
svi = SVI(
    model,
    m2_3,
    optim.Adam(1),
    Trace_ELBO(),
    avgfood=foxes_normalized_df.avgfood.values,
    groupsize=foxes_normalized_df.groupsize.values,
)
svi_result = svi.run(random.PRNGKey(0), 1000)
p2_3 = svi_result.params
post = m2_3.sample_posterior(random.PRNGKey(1), p2_3, (1000,))
post.pop('mu')
print_summary(post, 0.89, False)

100%|██████████| 1000/1000 [00:00<00:00, 2269.82it/s, init loss: 748.8373, avg. loss [951-1000]: 69.3732]



                mean       std    median      5.5%     94.5%     n_eff     r_hat
         a      0.00      0.04      0.00     -0.06      0.06    931.50      1.00
         b      0.90      0.04      0.89      0.84      0.97   1111.41      1.00
     sigma      0.43      0.03      0.43      0.39      0.48    946.68      1.00



Answer: food as a very strong effect on group size (b ~0.9), thus increasing food can substantially increases groupsize

Qn3. Reconsider the Table 2 Fallacy example (from Lecture 6), this time with an
unobserved confound U that influences both smoking S and stroke Y. 

First use the backdoor criterion to determine an adjustment set that allows
you to estimate the causal effect of X on Y, i.e. P(Y|do(X)). Second explain
the proper interpretation of each coefficient implied by the regression model
that corresponds to the adjustment set. Which coefficients (slopes) are causal
and which are not? There is no need to fit any models. Just think through
the implications